In [3]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import ee

ee.Initialize()

from sepal_ui.mapping import SepalMap
from sepal_ui.mapping import InspectorControl
from component.scripts.seplan import reduce_constraints
from component.model.recipe import Recipe
import component.parameter as cp
from sepal_ui.frontend.styles import SepalColor
from component.scripts.seplan import quintiles

SepalColor(**{f"col_{i}": col for i, col in enumerate(cp.final_viz["palette"])})
map_ = SepalMap()
inspector_control = InspectorControl(map_)
map_.add(inspector_control);

In [5]:
recipe = Recipe().load_model(admin="799")
# recipe = Recipe().load_model()

In [39]:
# I have to do this because is the view who is binding both models...
# I would rather to use the full feature collection object path
# or find a workaround to link them. FTM i'll let that as it is
aoi = recipe.seplan_aoi.feature_collection = recipe.seplan_aoi.aoi_model.feature_collection
aoi

In [56]:
seplan = recipe.seplan

In [57]:
benefit_index = seplan.get_benefit_index(clip=True)

In [58]:
masked_constraint_list = seplan.get_masked_constraints_list()

In [59]:
reduced_constraints = reduce_constraints(masked_constraint_list)

In [60]:
img_tree_cover_potential_mask = masked_constraint_list[0][0]

In [61]:
masked_index = seplan.get_constraint_index(clip=True)

In [13]:
benefits = [image for image, _ in seplan.get_benefits_list()]
quint_benefits = [quintiles(b, aoi) for b in benefits]
[
    [
        map_.addLayer(benefits[i].randomVisualizer(), {}, f"raw benefit {i}"),
        map_.addLayer(benefits[i].clip(aoi).randomVisualizer(), {}, f"benefit {i}"),
        map_.addLayer(quint_benefits[i].clip(aoi).randomVisualizer(), {}, f"quintile {i}"),
    ] for i, _ in enumerate(quint_benefits)
]

[[None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None]]

In [18]:
map_.centerObject(aoi);

In [14]:
map_.addLayer(benefit_index, {}, "benefit_index")

In [15]:
map_.addLayer(masked_index, cp.final_viz, "masked_index")

In [32]:
seplan.cost_model.assets

['projects/ee-amcmahon/assets/seplan/seplan_layers/features/opportunity_cost_20221110',
 'projects/john-ee-282116/assets/fao-restoration/features/AfCost_ha']

In [37]:
# unmask the images without normalizing as everything is in $/ha
images = [ee.Image(i).unmask() for i in seplan.cost_model.assets]

# create a normalized sum
norm_cost = ee.Image(0)
for v in images:
    norm_cost = norm_cost.add(v)

In [38]:
map_.addLayer(norm_cost, {}, "norm cost2")

In [16]:
map_

SepalMap(center=[0, 0], controls=(ZoomControl(options=['position', 'transparent_bg'], position='topright', wid…

In [46]:
map2 = SepalMap()
inspector_control = InspectorControl(map_)
map2.add(inspector_control);

In [47]:
map2.centerObject(aoi);
map2.addLayer(benefit_index, {}, "benefit_index")
map2.addLayer(masked_index, cp.final_viz, "masked_index")

In [48]:
map2

SepalMap(center=[11.456659096726085, 103.43142632380807], controls=(ZoomControl(options=['position', 'transpar…

In [55]:
map3 = SepalMap()
inspector_control = InspectorControl(map_)
map3.add(inspector_control);
map3.centerObject(aoi);
map3.addLayer(benefit_index, {}, "benefit_index")
map3.addLayer(masked_index, cp.final_viz, "masked_index")
map3

SepalMap(center=[0, 0], controls=(ZoomControl(options=['position', 'transparent_bg'], position='topright', wid…

In [62]:
map4 = SepalMap()
inspector_control = InspectorControl(map_)
map4.add(inspector_control);
map4.centerObject(aoi);
map4.addLayer(benefit_index, {}, "benefit_index")
map4.addLayer(masked_index, cp.final_viz, "masked_index")
map4

SepalMap(center=[0, 0], controls=(ZoomControl(options=['position', 'transparent_bg'], position='topright', wid…